In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

In [2]:
# Import API key and Configure Maps
from config import vacation_api_key

gmaps.configure(api_key=vacation_api_key)

In [3]:
#Use Final WeatherPy dataframe as start for Heatmap and Vacation Query
vacation_df = pd.read_csv("Output/weather_output.csv")
#Check dataframe column types
vacation_df.dtypes

Unnamed: 0      int64
City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Long          float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [4]:
#begin to create location coordinates and humidity for heatmap
locations = vacation_df[["Lat", "Long"]]
humidity = vacation_df["Humidity"].astype(float)                    

In [15]:
#Plotting Heatmap
fig = gmaps.figure(center=(50.0, -10.0), zoom_level=2)
#Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=1)
#Adding layer
fig.add_layer(heat_layer)
#Post Heatmap
fig


Figure(layout=FigureLayout(height='420px'))

SCREENSHOT of Heatmap above saved in /OUTPUT

In [24]:
#Begin to constrain vacation dataframe based on preferences
#I would like to go on a bike riding trip, I do not want it Windy or Hot.  Clouds are fine.
ideal_city_df = vacation_df.loc[(vacation_df["Wind Speed"] <= 7) & (vacation_df["Cloudiness"] < 10) & (vacation_df["Max Temp"] <= 75)].dropna()
ideal_city_df.head()                                                                                                        

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
20,20,jamestown,1,US,1599530033,93,42.10,-79.24,63.00,4.70
34,34,bredasdorp,0,ZA,1599530036,93,-34.53,20.04,50.00,2.24
36,36,mar del plata,0,ES,1599530036,87,43.40,-4.01,60.01,1.12
38,38,rodionovo-nesvetayskaya,0,RU,1599530037,50,47.61,39.71,55.40,2.24
40,40,pevek,0,RU,1599530037,78,69.70,170.31,42.06,3.13


In [25]:
#Narrow down the dataframe to include city, country, and coordinates.  Also adding Column for Hotels.
hotels_df = ideal_city_df.loc[:,["City", "Country", "Lat", "Long"]]
hotels_df["Hotel Name"] = ""
hotels_df.head()

,City,Country,Lat,Long,Hotel Name
20,jamestown,US,42.10,-79.24,
34,bredasdorp,ZA,-34.53,20.04,
36,mar del plata,ES,43.40,-4.01,
38,rodionovo-nesvetayskaya,RU,47.61,39.71,
40,pevek,RU,69.70,170.31,


In [26]:
#Begin to build the parameters for Google Search of Hotels
#setting up parameters
params = {
    "type" : "hotel",
    "keyword" : "hotel",
    "radius" : 5000,
    "key" : vacation_api_key
}
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
#Set up For loop to get Hotels that match city and Coordinates
for index, row in hotels_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]
    city_name = row["City"]
    
    